### Aruco sensor processing

Experimentation for the development of an Aruco tag based sensor processing system. 

In [1]:
import sys
sys.path.append("..")
from settings import Config
import numpy as np
import cv2
import torch
import pprint

from sensorprocessing import sp_aruco

# Move data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
if device.type == "cuda":
    print("Training on GPU")
else:
    print("Training on CPU")

Training on GPU


In [3]:
from sensorprocessing.aruco.utils import aruco_display

arucoDict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_100)
arucoParams = cv2.aruco.DetectorParameters()

In [4]:
# Specify the experiment
experiment = "sensorprocessing_aruco"
run = "aruco_128"
exp = Config().get_experiment(experiment, run)
pprint.pprint(exp)
sp = sp_aruco.ArucoSensorProcessing(exp, device)

Loading pointer config file: /home/ssheikholeslami/.config/BerryPicker/mainsettings.yaml
Loading machine-specific config file: /home/ssheikholeslami/SaharaBerryPickerData/settings-sahara.yaml
No system dependent experiment file
 /home/ssheikholeslami/SaharaBerryPickerData/experiments-Config/sensorprocessing_aruco/aruco_128_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: sensorprocessing_aruco/aruco_128 successfully loaded
{'MARKER_COUNT': 10,
 'data_dir': PosixPath('/home/ssheikholeslami/SaharaBerryPickerData/experiment_data/sensorprocessing_aruco/aruco_128'),
 'exp_run_sys_indep_file': PosixPath('/lustre/fs1/home/ssheikholeslami/BerryPicker/src/experiment_configs/sensorprocessing_aruco/aruco_128.yaml'),
 'group_name': 'sensorprocessing_aruco',
 'latent_size': 128,
 'run_name': 'aruco_128',
 'validation_demo': 'test',
 'validation_task': 'proprioception-aruco'}


In [5]:
camera_number = Config().values["robot"]["active_camera_list"][0]
print(camera_number)
#camera_number = 2
cap = cv2.VideoCapture(camera_number)

while(True):
    ret, frame = cap.read()
    if not ret:
        print("Capture frame returned False, exiting the loop")
        break
    frame = cv2.resize(frame, (256, 256))
    # imgresized = frame

    print(frame)

    #z = sp.process(frame)
    #print(z)

    # insert ARUCO code here

    marker_corners, marker_ids, rejected_candidates = cv2.aruco.detectMarkers(
        frame, arucoDict, parameters=arucoParams)

    detected_markers = aruco_display(marker_corners, marker_ids, rejected_candidates, frame)

    # end of ARUCO code

    # cv2.imshow('Press q to exit', frame)
    cv2.imshow('Press q to exit', detected_markers)
    ret = cv2.waitKey(1)
    if ret != -1:
        ret2 = ret & 0xFF
        if ret2 == ord('q'):
            break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

2
Capture frame returned False, exiting the loop


[ WARN:0@46.509] global cap_v4l.cpp:999 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@46.530] global obsensor_uvc_stream_channel.cpp:158 getStreamChannelGroup Camera index out of range


In [6]:
print(marker_corners)

NameError: name 'marker_corners' is not defined

In [6]:
marker_ids

In [7]:
frame.shape

(256, 256, 3)

This code creates the encoding for the Aruco markers because 

In [7]:
MARKER_COUNT = 10
XMAX = float(frame.shape[1])
YMAX = float(frame.shape[0])
NORMALIZER = np.tile([XMAX, YMAX], 4)

z = np.ones(MARKER_COUNT * (8 + 1)) * -1.0
for id, corners in zip(marker_ids, marker_corners):
    detection = corners[0].flatten() / NORMALIZER
    idn = id[0]
    z[idn * (8+1):(idn+1) * (8+1)-1] = detection


AttributeError: 'NoneType' object has no attribute 'shape'

In [8]:
z[0:9]

NameError: name 'z' is not defined